In [3]:
from cmath import exp
from numpy import log

class Neuron:
    def __init__(self, activation_function=lambda x: x, learning_rate=0.0000001) -> None:
        self.activation_function = activation_function
        self.learning_rate = learning_rate
        self.weights = []
    
    def setWeights(self, n:int):
        for _ in range(n):
            self.weights.append(np.random.randn())

    def setWeightsForInputLayer(self):
        self.weights = [1]

    def proceed(self, inputs):
        return self.activation_function(sum([inputs[i]*self.weights[i] for i in range(0, len(inputs))]))
    
    def balanceWeights(self, input, error):
        for i in range(0, len(self.weights)):
            self.weights[i] = self.weights[i] - error * self.learning_rate * input[i]
    
    def getErrorForOneBranch(self, error, branchIndex):
        totalWeights = sum([abs(self.weights[i]) for i in range(0, len(self.weights))])
        errorForOneBranch = abs(self.weights[branchIndex]/totalWeights) * error
        return errorForOneBranch
        

In [4]:
class NeuralNetwork:
    def __init__(self, layers, numberOfEpochs, activation_function=lambda x: x, learning_rate=0.0000001) -> None:
        self.layers = [[Neuron(activation_function, learning_rate) for i in range(0, layer)] for layer in layers]
        self.numberOfEpochs = numberOfEpochs

    def compute(self, inputLine):
        result = inputLine
        for layer in self.layers:
            result = [neuron.proceed(result) for neuron in layer]
        return result

    def balanceWeights(self, neuronIndex, error, inputLine, layerIndex):
        #back propagation
        self.layers[layerIndex][neuronIndex].balanceWeights(inputLine, error)

        if layerIndex == 0:
            return None

        for neuron in range(0, len(self.layers[layerIndex - 1])):
            self.balanceWeights(neuron,  self.layers[layerIndex][neuronIndex].getErrorForOneBranch(error, neuron), inputLine, layerIndex - 1)


    def train(self, input, output):
        self.layers = self.layers + [[Neuron(activation_function=lambda x: 1/(x + exp(-x).real)) for i in range(0, len(set(output)))]]
        
        for i in range(0, len(self.layers)):
            for neuron in self.layers[i]:
                if i == 0:
                    neuron.setWeights(len(input[0]))
                else:
                    neuron.setWeights(len(self.layers[i-1]))
        
        for epoch in range(0, self.numberOfEpochs):
            loss = 0
            for i in range(0, len(input)):
                inputLine = input[i]
                outputLine = output[i]
                computedOutput = self.compute(inputLine)
                if outputLine == "YES":
                    determined = 1
                    self.balanceWeights(0, 1 - computedOutput[0], inputLine, len(self.layers) - 1)
                    self.balanceWeights(1, 0 - computedOutput[1], inputLine, len(self.layers) - 1)
                else:
                    determined = 0
                    self.balanceWeights(0, 0 - computedOutput[0], inputLine, len(self.layers) - 1)
                    self.balanceWeights(1, 1 - computedOutput[1], inputLine, len(self.layers) - 1)
                loss = loss + determined * log(computedOutput[0]) + (1 - determined) * log(1 - computedOutput[0])
            loss = -1 * loss / len(input)
            print("Epoch ", epoch, " loss = ", loss, "\n")


    def predict(self, input):
        output = []

        for i in range(0, len(input)):
            computedOutput = self.compute(input[i])
            if computedOutput[0] > computedOutput[1]:
                output.append("YES")
            else:
                output.append("NO")
        return output



In [5]:
def clasificationPerformance(ground_truth, computed_values, positive_label):
    """
    Returneaza TN (True Negative), FP(False Positive), FN(False Negative), TP(True Positive)
    """
    TN = 0
    FP = 0
    FN = 0
    TP = 0
    
    for i in range(0, len(ground_truth)):
        #consideram malign = positive, benign = negative 
        if ground_truth[i] == positive_label:
            if computed_values[i] == positive_label:
                TP += 1
            else:
                FP += 1
        else:
            if computed_values[i] != positive_label:
                TN += 1
            else:
                FN += 1
    return TN, FP, FN, TP


def getAccuracy(TN, FP, FN, TP):
    """ 
    accuracy represents the overall performance of classification model:
    (TP+TN)/(TN+FP+FN+TP)
    """
    if (TN+FP+FN+TP) == 0:
        return 0
    return (TP+TN)/(TN+FP+FN+TP)

def getPrecision(FP, TP):
    """
    precision indicates how accurate the positive predictions are 
    TP/(TP+FP)
    """
    if (TP+FP) == 0:
        return 0
    return TP/(TP+FP)

def getRecall(TP, FN):
    """ 
    recall indicates the coverage of actual positive sample
    TP/(TP+FN)
    """
    if (TP+FN) == 0:
        return 0
    return TP/(TP+FN)


In [6]:
import numpy as np 
import matplotlib.pyplot as plt 
from PIL import Image
import pandas as pd

def readData() -> pd.DataFrame:
    dataFrame = pd.read_csv("datas.csv")
    dataFrame = dataFrame.dropna()
    return dataFrame


def getTrainingAndValidationSets():
    np.random.seed(5)
    dataFrame = readData()
    dataSize = dataFrame.shape[0]
    
    trainingIndexSet = np.random.choice(range(dataSize), size=int(0.7 * dataSize), replace=False)
    validationIndexSet = [i for i in range(dataSize) if i not in trainingIndexSet]

    trainingInputSet = [dataFrame["Photo"].iloc[index] for index in trainingIndexSet]
    trainingOutputSet = [dataFrame["Has Filter"].iloc[index] for index in trainingIndexSet]

    validationInputSet = [dataFrame["Photo"].iloc[index] for index in validationIndexSet]
    validationOutputSet = [dataFrame["Has Filter"].iloc[index] for index in validationIndexSet]

    return trainingInputSet, trainingOutputSet, validationInputSet, validationOutputSet


def getRGBValuesForAllImages(inputImages, size):
    rgbValues = []
    for imagePath in inputImages:
        rgbValues.append([])
        image = Image.open(imagePath)
        image = image.resize(size)

        for pixel in list(image.getdata()):
            r, g, b = pixel[0], pixel[1], pixel[2]
            rgbValues[-1].append(r)
            rgbValues[-1].append(g)
            rgbValues[-1].append(b)
    return rgbValues

def getClassifier(hiddenLayers, activationFunction, trainingInputs, trainingOutputs):
    neuralNetwork = NeuralNetwork(hiddenLayers, 10, activationFunction)  
    neuralNetwork.train(trainingInputs, trainingOutputs)
    return neuralNetwork


def testClassifier(hiddenLayers, activationFunction, size):
    trainingInputSet, trainingOutputSet, validationInputSet, validationOutputSet = getTrainingAndValidationSets()
    trainingInputs = getRGBValuesForAllImages(trainingInputSet, size)
    trainingOutputs = trainingOutputSet
    classifier = getClassifier(hiddenLayers, activationFunction, trainingInputs, trainingOutputs)

    validaitonInputs = getRGBValuesForAllImages(validationInputSet, size)
    outputs = classifier.predict(validaitonInputs)

    TN, FP, FN, TP = clasificationPerformance(validationOutputSet, outputs, "YES")
    accuracy = getAccuracy(TN, FP, FN, TP)
    precision = getPrecision(FP, TP)
    recall = getRecall(TP, FN)
    print("Accuracy: {}\nPrecision: {}\nRecall: {}".format(accuracy, precision, recall))

In [11]:
def tang_func(x):
    from numpy import tanh
    return tanh(x)

testClassifier([5,2,4], tang_func, size=(128,128))

Epoch  0  loss =  1.190048908892456 

Epoch  1  loss =  1.1385543450812687 

Epoch  2  loss =  1.087093423575307 

Epoch  3  loss =  1.0519376873811566 

Epoch  4  loss =  1.0288028528625315 

Epoch  5  loss =  1.0339967604026556 

Epoch  6  loss =  1.0391990852508268 

Epoch  7  loss =  0.9877994087267722 

Epoch  8  loss =  0.9587657163796528 

Epoch  9  loss =  0.9721722461325161 

Accuracy: 0.6296296296296297
Precision: 0.375
Recall: 0.375


In [15]:
testClassifier([3,3,3,3], tang_func, size=(64,64))

Epoch  0  loss =  1.199969768863418 

Epoch  1  loss =  1.1999645508161172 

Epoch  2  loss =  1.1999596937401693 

Epoch  3  loss =  1.2065136136707715 

Epoch  4  loss =  1.2065232582168106 

Epoch  5  loss =  1.206533299677878 

Epoch  6  loss =  1.2065438535780664 

Epoch  7  loss =  1.240092644036472 

Epoch  8  loss =  1.2400532252596561 

Epoch  9  loss =  1.233961823193132 

Accuracy: 0.2962962962962963
Precision: 1.0
Recall: 0.2962962962962963
